# Model Initialisation

In [1]:
from transformers import AutoTokenizer

In [2]:
model_ckpt = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Data Loading

In [3]:
filename = "Data.xlsx"

import pandas as pd

xls_data = pd.ExcelFile(filename)

In [4]:
train_data_df = pd.read_excel(xls_data, sheet_name="Sheet1")

In [5]:
train_data_df.columns = ["Questions", "Answers"]

In [6]:
import ace_tools_open as tools

In [7]:
tools.display_dataframe_to_user(name="Training DataFrame", dataframe=train_data_df)

Training DataFrame


In [8]:
sample_q_and_a = train_data_df.sample(n=1)
sample_question = sample_q_and_a["Questions"].iloc[0]
sample_answer = sample_q_and_a["Answers"].iloc[0]

In [9]:
print(f"Sample Question: {sample_question}\nSample Answer: {sample_answer}")

Sample Question: How many scales can I define?
Sample Answer: 23


In [10]:
print(f"Sample Question Type: {type(sample_question)}\nSample Answer Type: {type(sample_answer)}")

Sample Question Type: <class 'str'>
Sample Answer Type: <class 'int'>


# Answer Extraction

In [11]:
inputs = tokenizer(sample_question, str(sample_answer), return_tensors="pt")

In [12]:
print(tokenizer.decode(inputs["input_ids"][0]))

[CLS] how many scales can i define? [SEP] 23 [SEP]


In [13]:
import torch
from transformers import AutoModelForQuestionAnswering

In [14]:
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 4.4226, -5.1404, -5.5870, -5.8730, -5.9182, -6.0286, -5.7769, -6.6490,
          4.4226,  0.0339,  4.4225]]), end_logits=tensor([[ 4.9358, -5.9674, -5.3629, -4.9938, -5.7389, -5.7370, -5.7514, -5.2564,
          4.9358,  0.2823,  4.9358]]), hidden_states=None, attentions=None)


In [16]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [17]:
print(f"Input IDs shape: {inputs.input_ids.size()}")
print(f"Start logits shape: {start_logits.size()}")
print(f"End logits shape: {end_logits.size()}")

Input IDs shape: torch.Size([1, 11])
Start logits shape: torch.Size([1, 11])
End logits shape: torch.Size([1, 11])


In [18]:
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]
answer = tokenizer.decode(answer_span)
print(f"Question: {sample_question}")
print(f"Answer: {sample_answer}")

Question: How many scales can I define?
Answer: 23


In [19]:
from transformers import pipeline
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
pipe(question=sample_question, context=str(sample_answer), top_k=3)

Device set to use cpu


{'score': 0.00011576538236113265, 'start': 0, 'end': 2, 'answer': '23'}

In [20]:
# Initialize QA Pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use cpu


In [21]:
def chatbot():
    print("Chatbot: Hello! Ask me any question. Type 'exit' to stop.")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break
        
        # Try to find a matching question in the dataset
        match = train_data_df[train_data_df["Questions"].str.contains(user_input, case=False, na=False)]
        
        if not match.empty:
            response = match["Answers"].values[0]  # Return the first match
        else:
            # Use the transformer model for QA
            context = " ".join(train_data_df["Questions"].tolist())  # Combine all questions
            response = qa_pipeline({"question": user_input, "context": context})["answer"]
        
        print(f"Question asked: {user_input}")
        print(f"Chatbot: {response}")

# Run chatbot
chatbot()

Chatbot: Hello! Ask me any question. Type 'exit' to stop.
Question asked: How many tables can I have in my log?
Chatbot: 100
Question asked: Whats the maximum number of characters in a single text entry?
Chatbot: 250 / 32000 (varies per text type)
Question asked: Whats the maximum number of characters in a single text entry?
Chatbot: 250 / 32000 (varies per text type)
Chatbot: Goodbye!
